In [ ]:
!pip install langchain openai pandas
!pip install langchain[llms]
!pip install -U langchain-community

In [ ]:
import os
from getpass import getpass
OPENAI_API_KEY = getpass("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

OPENAI_API_KEY··········


In [ ]:
import pandas as pd
from langchain.evaluation.qa import CotQAEvalChain
from langchain.chat_models import ChatOpenAI

In [ ]:
# Upload CSV File
from google.colab import files
uploaded = files.upload()  # Manually select your CSV file

# Get the filename
csv_filename = list(uploaded.keys())[0]

Saving CSV-RAG.csv to CSV-RAG.csv


In [ ]:
# Load CSV into a DataFrame
# Read the CSV with a fallback encoding (ISO-8859-1 or Windows-1252)
df = pd.read_csv("CSV-RAG.csv", encoding="Windows-1252")

# Save it back as UTF-8
df.to_csv("CSV-RAG-UTF8.csv", index=False, encoding="utf-8")

In [ ]:
df.columns

Index(['Prompt', 'Ground Truth', 'RAG'], dtype='object')

In [ ]:
required_columns = {"Prompt", "Ground Truth", "RAG"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"CSV file must have columns: {required_columns}")

In [ ]:
# Initialize the language model (e.g., OpenAI's GPT-4)
llm = ChatOpenAI(model_name="gpt-4")

# Initialize the CotQAEvalChain
eval_chain = CotQAEvalChain.from_llm(llm=llm)

In [ ]:
# Prepare examples and predictions
examples = df.to_dict('records')
predictions = [{'result': row['RAG']} for _, row in df.iterrows()]

In [ ]:
# Evaluate the LLM's answers
results = eval_chain.evaluate(
    examples=examples,
    predictions=predictions,
    question_key='Prompt',
    context_key='Ground Truth',
    prediction_key='result'
)

In [ ]:
# Extract Grade for Text chain in Results

import re

def extract_grade(text):
    """Extracts the grade (CORRECT, INCORRECT, PARTIALLY CORRECT) from evaluation text."""
    match = re.search(r'GRADE:\s*(CORRECT|INCORRECT|PARTIALLY CORRECT)', text)
    return match.group(1) if match else "UNKNOWN"

df['evaluation_text'] = [result.get('text', "No reasoning provided") for result in results]
df['cotqa_grade'] = df['evaluation_text'].apply(extract_grade)

# Display Evaluation Results
display(df.head(5))  # Show only the first 10 rows to preview results

# Save Results to CSV
df.to_csv("evaluation_results.csv", index=False)
print("Evaluation results saved as 'evaluation_results.csv'.")

,Prompt,Ground Truth,RAG,evaluation_text,cotqa_grade
0,How many blood PK samples will be drawn per st...,29,A total of 29 blood PK samples will be drawn p...,The student's answer matches the context. The ...,CORRECT
1,Which third party vendors are being used in th...,"Austin Central Lab, Salus IRB, Medidata, Inoti...",1. Salus IRB 2. Medidata,The question asks for all third party vendors ...,INCORRECT
2,What is the main risk about urine PK for this ...,All urine must be collected during the study i...,The provided context does not explicitly menti...,The student's answer does not directly address...,INCORRECT
3,How often will the study team meet?,"Weekly during start-up and conduct, biweekly d...",The study team will have teleconferences with ...,The student's answer does not address the freq...,INCORRECT
4,How many Case Report Forms will be needed for ...,Approximately 77 CRFs per subject will be coll...,"Based on the information provided, each subjec...",The student's answer does not match the contex...,INCORRECT


Evaluation results saved as 'evaluation_results.csv'.


In [ ]:
df

In [ ]:
aggregation = df.groupby('cotqa_grade')
aggregation['cotqa_grade'].count()

,cotqa_grade
cotqa_grade,
CORRECT,35
INCORRECT,14
UNKNOWN,1
